請先在這裡定義 `n_pkt` 及 `m_byte` 分別為多少

In [ ]:
# 圖片大小
n_pkts = 7
m_bytes = 100

# 模型訓練參數
EPOCHS = 10
BatchSize = 256
Learning_Rate = 1e-4

圖片分類載入

In [ ]:
# 直接切換 NoteBook 的工作目錄，使用 !cd 只會切換終端機的
%cd /home/jinting/Desktop/fcu_topic/USTC-II
!pwd
!make
!./Convert_All_Flow_Ver_Colab.sh -2 /home/jinting/Desktop/2_Flow_Top_10000/2_Flow ../../DataSet {n_pkts}\
{m_bytes}
%cd ../../

In [ ]:
%cd /home/jinting/Desktop/

# Flow 圖片分類模型
n 個 CNN + 1 個 LSTM，將 n 張 PNG 圖輸入至 CNN，然後輸出序列送入 LSTM
分類為 26 類流量

## 1. 解壓縮＆資料集定義

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from pathlib import Path
from PIL import Image

In [ ]:
# 建立 Dataset
class FlowDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.class_map = {}
        self.transform = transform
        label_dirs = sorted(os.listdir(root_dir))

        for idx, label_name in enumerate(label_dirs):
            label_path = os.path.join(root_dir, label_name)
            if not os.path.isdir(label_path):
                continue
            self.class_map[label_name] = idx
            for session_folder in os.listdir(label_path):
                session_path = os.path.join(label_path, session_folder)
                if not os.path.isdir(session_path):
                    continue
                imgs = [os.path.join(session_path, f"{i}.png") for i in range(n_pkts)]
                if all(os.path.exists(p) for p in imgs):
                    self.samples.append((imgs, idx))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_paths, label = self.samples[idx]
        imgs = [Image.open(p).convert('L') for p in img_paths]
        if self.transform:
            imgs = [self.transform(im) for im in imgs]
        imgs = torch.stack(imgs)
        return imgs, label

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),  # 自動變成 shape: (1, 64, 64)
])

dataset = FlowDataset(os.path.join("./DataSet"), transform = transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
# 增加 workers 增加 Process 先將資料處理放到 RAM 中，使 GPU 載入資料速度變快，減少閒置
# 不過調整 workers 的數量可以在 GPU 的處理速度以及 RAM 的使用量中取得平衡
train_loader = DataLoader(train_set, batch_size = BatchSize, shuffle = True, num_workers = 4)
test_loader = DataLoader(test_set, batch_size = BatchSize, num_workers = 8)

## 2. 模型定義：n 個 CNN + LSTM

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1,32,3,1,1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,1,1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(64*16*16, 128)
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

class CNN_LSTM(nn.Module):
    def __init__(self, num_classes=26):
        super().__init__()
        self.cnns = nn.ModuleList([SimpleCNN() for _ in range(n_pkts)])
        self.lstm = nn.LSTM(input_size = 128, hidden_size = 64, batch_first = True)
        self.classifier = nn.Linear(64, num_classes)

    def forward(self, x):
        B = x.size(0)
        outs = []
        for i in range(n_pkts):
            outs.append(self.cnns[i](x[:,i]))
        seq = torch.stack(outs, dim=1)
        lstm_out, _ = self.lstm(seq)
        final = lstm_out[:, -1, :]
        return self.classifier(final)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN_LSTM().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = Learning_Rate)

# 調整使用 ReduceLROnPlateau，當 LRate 超過 2 epoch (patience) 不再下降，就 x 0.1 (factor)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.1, patience = 2)

## 3. 訓練 & 測試函式

In [ ]:
def train_epoch(loader):
    model.train()
    total_loss = 0
    total_correct = 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)
        total_correct += (outputs.argmax(1)==labels).sum().item()
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset)

def eval_model(loader):
    model.eval()
    total_loss = 0
    total_correct = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()*imgs.size(0)
            preds = outputs.argmax(1)
            total_correct += (preds == labels).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset), all_preds, all_labels

## 4. 開始訓練

In [ ]:
train_losses, train_accs = [], []
test_losses, test_accs = [], []
best_acc = 0
for ep in range(1, EPOCHS+1):
    tl, ta = train_epoch(train_loader)
    vl, va, _, _ = eval_model(test_loader)
    train_losses.append(tl); train_accs.append(ta)
    test_losses.append(vl); test_accs.append(va)
    scheduler.step(vl) # 根據 val_loss 來調整 LRate
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {ep}, train_loss={tl:.4f}, train_acc={ta:.4f}, test_acc={va:.4f}, now_lr={current_lr:.6f}")
    if va > best_acc:
        best_acc = va
        torch.save(model.state_dict(), 'best_model.pth')

## 5. 繪圖與混淆矩陣

In [ ]:
# Loss & Accuracy 圖
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.legend(); plt.title('Loss')
plt.subplot(1,2,2)
plt.plot(train_accs, label='Train Acc')
plt.plot(test_accs, label='Test Acc')
plt.legend(); plt.title('Accuracy')
plt.show()

# 混淆矩陣
_,_, preds, trues = eval_model(test_loader)
cm = confusion_matrix(trues, preds)
disp = ConfusionMatrixDisplay(cm)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
fig, ax = plt.subplots(figsize=(21,21))
disp.plot(ax=ax, xticks_rotation=90)
plt.show()

_,_, preds, trues = eval_model(test_loader)
cm = confusion_matrix(trues, preds)
plt.figure(figsize=(20, 9))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# 儲存模型到本地中
save_path = Path(f"./HAST-IDS-DataSet-Model/n{n_pkts}_m{m_bytes}_denoise")
save_path.mkdir(parents=True, exist_ok=True)

torch.save(model.state_dict(), f"{save_path}/cnn_lstm_hast_model_n{n_pkts}_m{m_bytes}.pth")
torch.save(model, f"{save_path}/cnn_lstm_complete_model_n{n_pkts}_m{m_bytes}.pth")

> [!note] 程式執行結束後，目前上面的混淆矩陣以及 notebook 並未存檔，如有需要，請自行存檔。